<a href="https://colab.research.google.com/github/amangupta2/ai_weather/blob/main/Compiling_AI_NWP_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
 # To use GPUs, under the Runtime Tab, change runtime type to T4 GPU
!nvidia-smi

Fri Apr 11 23:02:46 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   42C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# 1. Install Required Packages and Imports


In [ ]:
!pip install -q anemoi-inference[huggingface]==0.4.9 anemoi-models==0.3.1
!pip install -q earthkit-regrid==0.4.0 ecmwf-opendata
!pip install -q flash_attn
!pip install -q matplotlib
!pip install -q cartopy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.7/42.7 kB 3.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 13.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.7/97.7 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.3/65.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 kB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 101.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import datetime
from collections import defaultdict

import numpy as np
import earthkit.data as ekd
import earthkit.regrid as ekr

from anemoi.inference.runners.simple import SimpleRunner
from anemoi.inference.outputs.printer import print_state

from ecmwf.opendata import Client as OpendataClient

# 2. Retrieve Initial Conditions from ECMWF Open Data

In [14]:
# List of parameters to retrieve from ECMWF open data

PARAM_SFC = ["10u", "10v", "2d", "2t", "msl", "skt", "sp", "tcw", "lsm", "z", "slor", "sdor"]
PARAM_SOIL =["vsw","sot"]
PARAM_PL = ["gh", "t", "u", "v", "w", "q"]
LEVELS = [1000, 925, 850, 700, 600, 500, 400, 300, 250, 200, 150, 100, 50]
SOIL_LEVELS = [1,2]

In [18]:
DATE = OpendataClient().latest() # this date is used to access the ECMWF server
#DATE = datetime.datetime(2020,1,1,0,0,0)
print("Initial date is", DATE)

Initial date is 2025-04-11 12:00:00


In [19]:
# Get the data from the ECMWF Open Data API and create initial state

def get_open_data(param, levelist=[]):
    fields = defaultdict(list)
    # Get the data for the current date and the previous date
    for date in [DATE - datetime.timedelta(hours=6), DATE]:
        data = ekd.from_source("ecmwf-open-data", date=date, param=param, levelist=levelist)
        for f in data:
            # Open data is between -180 and 180, we need to shift it to 0-360
            assert f.to_numpy().shape == (721,1440)
            values = np.roll(f.to_numpy(), -f.shape[1] // 2, axis=1)
            # Interpolate the data to from 0.25 to N320
            values = ekr.interpolate(values, {"grid": (0.25, 0.25)}, {"grid": "N320"})
            # Add the values to the list
            name = f"{f.metadata('param')}_{f.metadata('levelist')}" if levelist else f.metadata("param")
            fields[name].append(values)

    # Create a single matrix for each parameter
    for param, values in fields.items():
        fields[param] = np.stack(values)

    return fields

# Get input fields
fields = {}

# Add the single levels fields
fields.update(get_open_data(param=PARAM_SFC))
soil=get_open_data(param=PARAM_SOIL,levelist=SOIL_LEVELS)
# Soil parameters have been renamed since training this model, we need to rename to the original names
mapping = {'sot_1': 'stl1', 'sot_2': 'stl2',
           'vsw_1': 'swvl1','vsw_2': 'swvl2'}
for k,v in soil.items():
    fields[mapping[k]]=v

# Add the pressure level fields
fields.update(get_open_data(param=PARAM_PL, levelist=LEVELS))

# Convert geopotential height into geopotential
# Transform GH to Z
for level in LEVELS:
    gh = fields.pop(f"gh_{level}")
    fields[f"z_{level}"] = gh * 9.80665

# Create Initial State
input_state = dict(date=DATE, fields=fields)



<multiple>:   0%|          | 0.00/7.24M [00:00<?, ?B/s]

9533e90f8433424400ab53c7fafc87ba1a04453093311c0b5bd0b35fedc1fb83.npz:   0%|          | 0.00/7.11M [00:00<?, ?B…

<multiple>:   0%|          | 0.00/7.27M [00:00<?, ?B/s]

20250411060000-0h-scda-fc.grib2:   0%|          | 0.00/2.12M [00:00<?, ?B/s]

20250411120000-0h-oper-fc.grib2:   0%|          | 0.00/2.13M [00:00<?, ?B/s]

<multiple>:   0%|          | 0.00/55.0M [00:00<?, ?B/s]

<multiple>:   0%|          | 0.00/55.1M [00:00<?, ?B/s]

# 3. Load the Model and Run the Forecast

In [20]:
# Download the Model's Checkpoint from Hugging Face & create a Runner
checkpoint = {"huggingface":"ecmwf/aifs-single-1.0"}
runner = SimpleRunner(checkpoint, device="cuda")


In [22]:
# Run the forecast
for state in runner.run(input_state=input_state, lead_time=12):
    print_state(state)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

encoder_graph.jpeg:   0%|          | 0.00/168k [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.59k [00:00<?, ?B/s]

aifs-single-mse-1.0.ckpt:   0%|          | 0.00/994M [00:00<?, ?B/s]

...

config_pretraining.yaml:   0%|          | 0.00/10.9k [00:00<?, ?B/s]

scorecard_single1.0_vs_ifs_2024.png:   0%|          | 0.00/271k [00:00<?, ?B/s]

run_AIFS_v1.ipynb:   0%|          | 0.00/315k [00:00<?, ?B/s]

aifs_diagram.png:   0%|          | 0.00/470k [00:00<?, ?B/s]

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [ ]:
!pip install -q matplotlib
!pip install -q cartopy

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.tri as tri